# Workshop 1
Hi and welcome to the first of a series of workshops on using MODFLOW-6 with Flopy.
To complete this workshop you will need the following:
1. The USGS Modflow-6 suite - get it [here](https://water.usgs.gov/water-resources/software/MODFLOW-6/mf6.4.2.zip)
2. A Python environment with flopy, numpy, pandas, os, shutil, sys and matplotlib

Ideally the Modflow6 execuatble (located in the bin folder of the downloaded suite) should be avaialble system wide otherwise it needs to be placed in the folder where this notebook will running.

Open the mf6io.pdf document that comes with the MF6 suite in the documentation folder. This is the reference document that you should be using to check your model input files and it explains all the different options available to you. This point needs to be emphasized. As the modeller, ***it is your responsibility to check your input files***. Do not presume that because Flopy executed without error that your input files are fine. Flopy is a tool to help you create those files – it can still produce input files that will cause your model to fail.

Only the base flopy installation is used - not the optional dependencies.

This notebook was developed with flopy version 3.4.2

We'll get straight into it.

# Imports
These are some of the most common libraries that I use when working on building a model with Flopy. You will notice that some libraries are imported with an alias. Hopefully you are all familiar with the reasons for this. If not don't worry it will become clearer later. I find it useful to always check which version of the library I am using in case I get any issues with errors that may be specific to that version.

In [ ]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import flopy


print(f"Pandas version = {pd.__version__}")
print(f"Numpy version = {np.__version__}")
print(f"Flopy version = {flopy.__version__}")

# Setting up some folders to dump things in later
We will be creating files and perhaps using files from other utilities, so it is nice to have things organised. You can do this however you want but as a simple example you can follow what I do below. Let’s first look at where we are on the computer by using the os library.

In [ ]:
os.getcwd()

The path above is where we are working right now. Lets make a new folder specific to this workshop.

In [ ]:
ws1 = os.path.join('workshop_1') # here we are making a path not creating the folder
if os.path.exists(ws1): # here we are asking if the path exists on the computer. 
    shutil.rmtree(ws1)# if it does exist, delete it and all the files in it
    os.mkdir(ws1) # then remake it
else:
    os.mkdir(ws1) # if it doesn't exist then make the folder

Let's make a few extra paths to folders that we will use later. Notice how the workshop_1 folder path (which is ws1) is now the first entry in the join chain. This means that the paths we are making will be sub-directories of the workshop_1 folder.

In [ ]:
gis_f = os.path.join(ws1,'GIS') # creating a sub-directory path for our gis input/output
model_f = os.path.join(ws1,'model') # creating a sub-directory path for our model input/output
plots_f = os.path.join(ws1,'plots') # creating a sub-directory path for our plots

In [ ]:
# take a look at the paths we made
print(gis_f)
print(model_f)
print(plots_f)

Now we will make a few sub-directories inside our workshop folder by using a for loop from a list of strings called subdirectory_list. Once you execute the code below check you folder for the subdirectories.

In [ ]:
subdirectory_pathlist = [gis_f,model_f,plots_f] # this is our list of sub-directory paths, each one is a path string
for path in subdirectory_pathlist: # for each path in the list above
    if os.path.exists(path): # check if the folder already exists
        pass # if it does exist then do nothing
    else:
        os.mkdir(path) # otherwise make the folder

# Setting up a MF6 simulation
The simulation namefile is required for all MF6 modelling work. What you are creating next is the mfsim.nam file. This is the file at the highest level when it comes to all modelling work in you specific project using MF6. The mfsim.nam file is what the MF6 executable will look for as soon as you try to run a model in you project folder.

In [ ]:
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f) # this one is mandatory if you want to make sure that the model files are created in the folder we want

In [ ]:
sim

Let's see if we can write the mfsim.nam file using Flopy's package write function.

In [ ]:
sim.write_simulation()

Hmm, looks like it failed trying to find the tdis package object but it did succeed in writing the mfsim.nam file before failing. The write_simualtion() method expects a timing package object to be present or it will fail. We'll discuss this more later but for now let's take a quick peek at the mfsim.nam file. You can execute the code below to view the contents of the file or alternatively open the file in its folder with your favourite text editor. Do you know where to look for the file?

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"mfsim.nam"))]

Open the mf6io.pdf document to see what else could be in this file if we did things a bit differently.
We will discuss what the other sections are in the document and whether you add them explicitly to the sim object of if Flopy will handle that for you.

Things to think about.

Can I remake the object? Can a simulation have more than one simulation object? Are there any conflicts with options? What happens if I enter something incorrectly? Try to remake the sim object with all options turned on.

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f, # this one is mandatory if you want to make sure that the model files are created in the folder we want
                             continue_=True,
                             nocheck=True,
                             lazy_io=True,
                             memory_print_option="ALL",
                             write_headers=False) 

In [ ]:
sim.write_simulation()

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"mfsim.nam"))]

Wait a minute what about maxerrors?

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f, # this one is mandatory if you want to make sure that the model files are created in the folder we want
                             continue_=False,
                             nocheck=False,
                             lazy_io=False,
                             memory_print_option="ALL",
                             write_headers=False,
                             maxerrors=20) 

Out of options. Time to go to GitHub or dig into the source code of the library installed on your computer. What you will find is that the installed version does not have the maxerrors option available but the online develop branch version does. This is likely to change in the future. This does not mean you can't use the option because it already exists in MF6. It does mean that you can't use it through Flopy because it is in development. For the purposes of this workshop we don't need it but this brief diversion is handy to demonstrate that the software you are using is being developed on a continuous basis. This includes both MF6 and Flopy. New features added to MF6 may take a while to become available in Flopy. So you need to know how to edit the input files manually, which is where the mf6io.pdf becomes your primary resource. 

What if I want to change something in the sim object after it has already been created. There are methods that allow you to access settings directly but for these workshops we're going to keep it as simple as possible so just recreate the object with new settings. but be sure to keep the object names the same. Otherwise, you may not be replacing the previous object but attempting to add another one.

In [ ]:
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f) # this one is mandatory if you want to make sure that the model files are created in the folder we want

Thats basically all you need to know for the sim object in order to create it. But what if you have an existing model and you want to load it instead. The Flopy GitHub introduction model is available as in import for use with the sim package that you just created. Execute the following block of code to import the function and then run it by passing in your sim object.

In [ ]:
from helpers import ws1_mod

In [ ]:
ws1_mod(sim)

So you've just built a complete model. Take a quick look at the sim object now. Then open up the simulation workspace folder. Note that you haven't written your model files yet. Let's take a look at what the sim object looks like now.

In [ ]:
sim

Now to write your simulation.

In [ ]:
sim.write_simulation()

Have a look at your model folder now. The model input files should be there - but no output files just yet.
Now lets run the simulation.

In [ ]:
sim.run_simulation()

Have a look at the folder again. If your model completed successfully. You should now see output files in addition to your input files. Great now we can look at how to load an existing model using the simulation object.

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=model_f) 
# note that for MF6 you generally only have to provide the path to the model
# It will automatically search for the mfsim.nam file in that location 
# then load the model from that.

If the model was not created by Flopy but some other GUI then you may have trouble loading all packages.
Again this is where you need the mf6io.pdf to help you out because you may be able to load the package with a minor tweaks to the problematic input file.
If it requires major modifications then perhaps consuder a load without that package using the load only options as demonstrated below.

In [ ]:
load_list = ["npf"]
sim = flopy.mf6.MFSimulation.load(sim_ws=model_f, load_only=load_list) 
#note that there are certain packages that it will always load irrespective of what you declare as load only.

Once you load the model and whatever packages you want, then you can "get" the simulation level package objects from the simulation object. Note only simulation level package objects. Accessing loaded model level package objects requires that you first "get" the loaded model object from the loaded simulation. We will do more with the model object later but for completeness we will access both simulation and model level package objects here.

In [ ]:
tdis = sim.get_package('tdis') # a simulation level package object
gwf = sim.get_model("MyModel") # get the model object in order to access model level package
npf = gwf.get_package("NPF") # get the loaded npf package object and assign it to a variable

In [ ]:
# check that the variable type mtaches the loaded package
type(npf)

Now we are going to clear all that and start again but this time we will make our own time discretization package.

In [ ]:
for x in [sim,gwf,npf,tdis]: # loop through a list of objects we want to remove
    del x # use the del statement to remove the objects, this clears them from memory
flist = [x for x in os.listdir(model_f)] # gets a list of all the files in the model folder
_ = [os.remove(os.path.join(model_f,f)) for f in flist] # removes all the files in the model folder, faster as list comprehension? Maybe.
# now check that it worked
flist = [x for x in os.listdir(model_f)] # get the files again
flist # should see an empty list

# The timing file or TDIS

The important thing to note with the creation of the tdis object is that you are passing in the simulation object. Only the simulation level package objects require that you pass in a sim object.


In [ ]:
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f) # this one is mandatory if you want to make sure that the model files are created in the folder we want
tdis = flopy.mf6.ModflowTdis(sim) # will give you a basic tdis object set for single steady-state stress period

In [ ]:
tdis

That may be all you need but let’s try to make a slightly better one specifying time units and our model start datetime as ISO 8601 format. Note also that it will warn you that it detects an existing timing package.

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00")
tdis

In [ ]:
sim.write_simulation()

What if we have a transient model? Lest start with a transient single stress period model. Its going to run for a year and have approximately weekly time steps.

In [ ]:
# start by creating a new tuple for the first stress period
# the tuple needs to be in a sequence of stress period length, number of steps and mutiplier.
# you can change it directly or rebuild the object
tdis.perioddata.array[0] = (365.25,52,1)
tdis
sim.write_simulation()
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.tdis"))]
#tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00")

In [ ]:
# alternativley just rebuild it
del tdis
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00",perioddata=[(365.25,52,1)])
tdis

Okay now we will try something a bit tricky with time discretization. We are going to build our stress period data using actual dates. We want an initial steady-state period followed by 20 years of monthly stress periods with monthly time steps followed by a single stress period of 100 years with approximately monthly time steps. Our model will start on the 31/12/2023 with the steady-state period of 1 day. This means we want our transient period to start on the 01/01/2024.

In [ ]:
# we start with the 20 years of monthly stress periods, note the starting date used in the range.
dates = pd.date_range('2024-01-01','2043-01-01', freq='MS').tolist() # use pandas to create a list of timestamps in monthly frquency
dates

In [ ]:
perlens = [(dates[x]-dates[x-1]).days for x in range(1,len(dates))] # gives me the monthly period lengths in days. note use of .days
perlens # this will help to check if we have the correct starting values.
# Question: why do I start my range at 1 and not zero? Why not for x in dates
# Recall that we need to provide a list of tuples to the tdis object where the first item in the tuple is the stress period length.

In [ ]:
# check the number of entries in period lengths and assgn it to a variable because we will need it later
numper = len(perlens) # checks the length of the list. It should be 228 because we have 20 * 12 stress periods.

In [ ]:
stp = 1 # we'll see how the model stability goes with 1 time step per period to start off with 
# build the transient stress period data tuples for the tdis [ackage
_ = [(x,stp,1) for x in perlens] # note the final 1 in the tuple is the multiplier. We won't use it to begin with.

In [ ]:
# add in the steady state stress period at the start as (1,1,1) and 100 year period at the end
# note my 100 year period has 1200 steps so sticking with approximatley monthly time steps
pdata = [(1,1,1), *_, (36525, 1200, 1)] # note the use of unpacking the _ tuple list
# now we have a single recovery period that is 100 years in length with approximatley mothly time steps
numper = len(pdata) # we are updating the value of numper here to include our steady-state and 100 year periods
print(numper) # print the value for viewing
pdata # view all the period data. Note that this is what we will provide to the 

As a reference we are going to create a dataframe for use later. This will provide a check for any transient behaviour for boundary conditions. For example, if we know pumping is scheduled to commence in February 2025 then we might need to know which stress period this is. I say might here because there are alternative ways to control transient boundary conditions in MF6 that were not available in previous versions of MODFLOW.

In [ ]:
ss = pd.Timestamp("2023-12-31") # set the date for the SS period
dates = [ss,*dates] # add the date
# Hmm, could we have done this originally and included the steady-state stress period when we got the tuple lengths? Try it yourself.
len(dates)

In [ ]:
# Now lets build the dataframe
df = pd.DataFrame() # start with en empty dataframe
df['Date'] = dates # create a column with dates in it
df['SP'] = range(1,len(dates)+1) # create a column with the actual MF6 stress period number in it
df['Flopy_SP'] = range(len(dates)) # create a column with the Flopy zero-based stress period numbers in it
df.to_csv('model_timing.csv',index=None) # export the csv to the project folder
# if you want to keep this dataframe available for use later in the script then make a copy of it using a descriptive name
modtime_df = df.copy()

In [ ]:
# Now we create the new tdis object
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00",
                             perioddata=pdata,nper=numper)
tdis

In [ ]:
# lets write our simulation
sim.write_simulation()

In [ ]:
# lets look at the tdis file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.tdis"))]

In [ ]:
from helpers import ws1_mod_trans 
# now we are importing the same model as before but with one extra package being built. 
# What is the new package? Why is it needed now?

In [ ]:
# now lets run the transient version of the same model 
ws1_mod_trans(sim,tdis) # remember this is only putting the objects in memory you havent created the files yet

In [ ]:
sim.write_simulation() # write the simulation files
sim.run_simulation() # run the simulation

Ideally you should be building models that are very stable but in some instances you may have to use an unstable model or you have advanced stress packages that are causing you some strife so you may opt to use adaptive time stepping. This will automatically change the time step length in an attempt to solve the model for you. Note this is one strategy to get to a solution for your model there are other options too. You may also choose to set this up immediately from the get go but there are some caveats to doing this with regards to model output that you should be aware of. Take some time to read the ATS section in the mf6io.pdf to make sure that you understand how it all works. I'll provide some brief descriptions below.

In [ ]:
# We need to provide the tdis object with some data for the ATS package in order for it to be created for us.
# in this example we are only going to want ATS active during the monthly stress periods
# first we define which stress perios we want the ATS active for
iperats = [x for x in range(1,numper-1)] # what am i doing here exactly?
dt0 = 0 # Now we say what we want the initial time step for the stress period to be. An entry of zero is special. Read up why.
dtmin = 0.01 # next we need to define the minimum time step length we will allow. Recall our time units are days.
# This next one requires the most thought and depending on how much your model is struggling to converge may require adjusting.
# We will use a recommended value for now
dtadj = 3.0
# The last variable is also worth reading up about, it controls how the time step length is reduced when your model fails to converge
# for brevity we will use the recommended value
dtfailadj = 5.0
# now we create our input data list of tuples. Check the mf6io.pdf for the entries to make sure we have them ordered correctly.
ats_pdata = [[x,dt0,dtmin,y,dtadj,dtfailadj] for x,y in zip(iperats,perlens)] # What are x and y here?
# now we need to pass this on to the tdis package and re-write

In [ ]:
# now we initialize the ats in the existing tdis object
tdis.ats.initialize(maxats=len(ats_pdata),perioddata=ats_pdata)

In [ ]:
sim.write_simulation() # look for the ats 

In [ ]:
# lets look at our tdis input file now
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.tdis"))]

In [ ]:
# lets look at our ATS input file now
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.tdis.ats"))]

In [ ]:
# Lets run it
sim.run_simulation()

In [ ]:
# lets clear everything again
for x in [sim,tdis]: # loop through a list of objects we created that we want to remove
    del x # use the del statement to remove the objects, this clears them from memory
flist = [x for x in os.listdir(model_f)] # gets a list of all the files in the model folder
_ = [os.remove(os.path.join(model_f,f)) for f in flist] # removes all the files in the model folder, faster as list comprehension? Maybe.
# now check that it worked
flist = [x for x in os.listdir(model_f)] # get the files again
flist # should see an empty list

# The solver or IMS package

This package is mandatory. But the input file is reasonably simple to construct with Flopy. There are lots of options for you to use if you really want to customise the solver but the defaults are generally fine to ensure convergence. The most common settings that you will change are your convergence criteria and you number of iterations. There may be occasions where you want to optimise your solver for speed then the other settings could be of benefit but if you stick with the "SIMPLE", "MODERATE" and "COMPLEX" you should be fine. If not, consider investigating model design issues first before resorting to exotic solver configurations just to converge. The mf6io.pdf document has some information on the benefit of each solver setting plus recommendations for values. These are worth a read at least once. You have to have a sim object in memory before you create this object because it is needed as part of the input.

In [ ]:
# Lets create our sim and tdis objects again we'll drop ATS for now. But stick with the transient model configuration.
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", 
                             verbosity_level=1,
                             sim_ws=model_f)
dates = pd.date_range('2024-01-01','2043-01-01', freq='MS').tolist()
ss_date = pd.Timestamp("2023-12-31")
dates = [ss_date,*dates] # using all the dates
perlens = [(dates[x]-dates[x-1]).days for x in range(1,len(dates))]
stp = 1
_ = [(x,stp,1) for x in perlens] # now they all have a step of 1 but we want 1200 steps for the last one
pdata = [*_,(36525,1200,1)] # adding the final stress period as a tuple
numper = len(pdata)
# Could we have added the final date to our dates and then just changed the number of time steps?
# Look up mutable and imutable objects in Pyhton to find out why.
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00",
                             perioddata=pdata,nper=numper)

The ims package object below is an example. As you can see the settings are very straight forward. Most of the input variables are either a sinlge string, single integer or float variable. 

In [ ]:
ims = flopy.mf6.ModflowIms(sim, complexity='MODERATE', # using the moderate solver settings option Note passing in sim object
                           csv_inner_output_filerecord='inner.csv', # asking to output a csv of the inner iterations
                           csv_outer_output_filerecord='outer.csv', # asking for an output of the outer itertions
                           outer_maximum=500, # Overiding the default MODERATE value (50) setting the maximum number of outer iterations
                           inner_maximum=500, #  Overiding the default MODERATE value (100) setting the maximum number of inner iterations
                           outer_dvclose=0.01, # This is the default MODERATE value setting the convergence criteria for the outer iteration
                           inner_dvclose=0.001) #  Overiding the default MODERATE value (0.01) setting the convergence criteria for the inner iteration

In [ ]:
# let's write the simulation
sim.write_simulation()

In [ ]:
# Let's look at the file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.ims"))]

In [ ]:
# import a new model that needs the sim, tdis and ims objects
from helpers import ws1_mod_trans2

In [ ]:
# now run the imported function and pass
ws1_mod_trans2(sim,tdis,ims) # recall we are only building package objects here not writing files.

In [ ]:
# now write the files then check you model folder
sim.write_simulation()

In [ ]:
# run the simulation
sim.run_simulation()

Now we should have some information on the model convergence behaviour in two csv files named "inner.csv" and "outer.csv". Take a look at them and see if you can decipher how your chosen convergence criteria influence the solver behaviour.

In [ ]:
# lets clear everything again
for x in [sim,tdis,ims]: # loop through a list of objects we created that we want to remove
    del x # use the del statement to remove the objects, this clears them from memory
flist = [x for x in os.listdir(model_f)] # gets a list of all the files in the model folder
_ = [os.remove(os.path.join(model_f,f)) for f in flist] # removes all the files in the model folder, faster as list comprehension? Maybe.
# now check that it worked
flist = [x for x in os.listdir(model_f)] # get the files again
flist # should see an empty list

# The model name-file
This is the last simulation level package object that you need to create and is also generally very straight forward to build. It is at this point that you need to provide a model-name. Recall that we set a simulation name previously and a simulation folder to write our model files in. With the model object you can change where the model level files are written using a relative path (optional). The example below is generally all you need. The use of newtonoptions is probably the only thing that requires some consideration. 

In [ ]:
# first we build our simulation from scratch again (I'm channeling Zed Shaw here)
# Lets create our sim and tdis objects again we'll drop ATS for now. But stick with the transient model configuration.
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", 
                             verbosity_level=1,
                             sim_ws=model_f)
dates = pd.date_range('2024-01-01','2043-01-01', freq='MS').tolist()
ss_date = pd.Timestamp("2023-12-31")
dates = [ss_date,*dates] # using all the dates
perlens = [(dates[x]-dates[x-1]).days for x in range(1,len(dates))]
stp = 1
_ = [(x,stp,1) for x in perlens] # now they all have a step of 1 but we want 1200 steps for the last one
pdata = [*_,(36525,1200,1)] # adding the final stress period as a tuple
numper = len(pdata)
# Could we have added the final date to our dates and then just changed the number of time steps?
# Look up mutable and imutable objects in Pyhton to find out why.
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00",
                             perioddata=pdata,nper=numper)
ims = flopy.mf6.ModflowIms(sim, complexity='MODERATE', # using the moderate solver settings option Note passing in sim object
                           csv_inner_output_filerecord='inner.csv', # asking to output a csv of the inner iterations
                           csv_outer_output_filerecord='outer.csv', # asking for an output of the outer itertions
                           outer_maximum=500, # Overiding the default MODERATE value (50) setting the maximum number of outer iterations
                           inner_maximum=500, #  Overiding the default MODERATE value (100) setting the maximum number of inner iterations
                           outer_dvclose=0.01, # This is the default MODERATE value setting the convergence criteria for the outer iteration
                           inner_dvclose=0.001) #  Overiding the default MODERATE value (0.01) setting the convergence criteria for the inner iteration

We will discuss some of the other options here. They tend to be provided for flexibility and are more use case specific. The example should still provide sufficient guidance on how to use the different options.

In [ ]:
# Now first set the model name
model_name = 'flow' 
# build the object without a specified relative path for the model files
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name, save_flows=True, newtonoptions="under_relaxation")
# The save flows option just indicates that we want to save a budget file
# What is the budget file? Do you need it? 
# This is worth thinking about because it can make a big different to your model speed.

In [ ]:
# now look at the sim object and note the inclusion of the model in the sim at the bottom of the output.
sim

In [ ]:
# Now write the simulation.
sim.write_simulation()

In [ ]:
# Let's look at the model name-file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"flow.nam"))]

In [ ]:
# Look at the simulation name-file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"mfsim.nam"))]

In [ ]:
# import a new model that needs the sim, tdis, ims and gwf objects plus the model name
from helpers import ws1_mod_trans3

In [ ]:
# now run the imported function and pass in the required objects and variables
ws1_mod_trans3(sim,tdis,ims,gwf,model_name) # recall we are only building package objects here not writing files.

In [ ]:
# write the model files
sim.write_simulation()

In [ ]:
# run the model and check the model directory for the output files
sim.run_simulation()

In [ ]:
# Let's clear all the files from the model folder
shutil.rmtree(model_f) # remove the model folder (this will also remove subdirectories in the folder)
os.mkdir(model_f) # remake a new model folder
# This is a different way to get rid of the files. 

# Now repeat with a new option

Here we use a relative path for the model which will place the model specific input files in a subdirectory. Keep an eye on your files and folders when you write the simualtion to get a feel for what this does. Also have a look at the mfsim.nam file to see how the path to the modle name fiel is altered. There is also a slight change to how we go about creating the tdis object's period data. Can you see where?

In [ ]:
# first we build our simulation from scratch again (I'm channeling Zed Shaw here)
# Lets create our sim and tdis objects again we'll drop ATS for now. But stick with the transient model configuration.
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", 
                             verbosity_level=1,
                             sim_ws=model_f)
dates = pd.date_range('2024-01-01','2043-01-01', freq='MS').tolist()
ss_date = pd.Timestamp("2023-12-31")
dates = [ss_date,*dates] # using all the dates
perlens = [(dates[x]-dates[x-1]).days for x in range(1,len(dates))]
stp = 1
_ = [(x,stp,1) for x in perlens] # now they all have a step of 1 but we want 1200 steps for the last one
pdata = [*_,(36525,1200,1)] # adding the final stress period as a tuple
numper = len(pdata)
# Could we have added the final date to our dates and then just changed the number of time steps?
# Look up mutable and imutable objects in Pyhton to find out why.
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00",
                             perioddata=pdata,nper=numper)
ims = flopy.mf6.ModflowIms(sim, complexity='MODERATE', # using the moderate solver settings option Note passing in sim object
                           csv_inner_output_filerecord='inner.csv', # asking to output a csv of the inner iterations
                           csv_outer_output_filerecord='outer.csv', # asking for an output of the outer itertions
                           outer_maximum=500, # Overiding the default MODERATE value (50) setting the maximum number of outer iterations
                           inner_maximum=500, #  Overiding the default MODERATE value (100) setting the maximum number of inner iterations
                           outer_dvclose=0.01, # This is the default MODERATE value setting the convergence criteria for the outer iteration
                           inner_dvclose=0.001) #  Overiding the default MODERATE value (0.01) setting the convergence criteria for the inner iteration

In [ ]:
# Now first set the model name
model_name = 'flow' 
# build the object again this time with a specified relative path for the model files
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name, 
                           save_flows=True, 
                           newtonoptions="under_relaxation",
                           model_rel_path='.\\flow')

In [ ]:
# lets take a look at the sim object again
sim

In [ ]:
# import a new model that needs the sim, tdis, ims and gwf objects plus the model name
from helpers import ws1_mod_trans3

In [ ]:
# now run the imported function and pass in the required objects and variables
ws1_mod_trans3(sim,tdis,ims,gwf,model_name) # recall we are only building package objects here not writing files.

In [ ]:
# write the model files
sim.write_simulation()

In [ ]:
# run the model and check the model directory for the output files
sim.run_simulation()

# Thats all folks

Today we covered the main simualtion level packages. Next session we will focus purley on spatial discretisation. Hope it was worthwhile.